In [1]:
#由于cifar10.py中有argparse函数，Jupyter不支持，所以不能导入
import cifar10_input
import tensorflow as tf
import numpy as np
import time

### 1、数据处理

In [2]:
max_steps = 300
batch_size = 128
data_dir = 'tmp/cifar-10-batches-bin'

In [3]:
#产生训练需要的数据集，返回的是tensor。并且distorted函数已经对图片做了随机的数据增强处理，包括反转剪裁……
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir,
                                                            batch_size=batch_size)


Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [4]:
print images_train
print labels_train

Tensor("shuffle_batch:0", shape=(128, 24, 24, 3), dtype=float32)
Tensor("Reshape_1:0", shape=(128,), dtype=int32)


In [5]:
#生成测试数据，不用随机处理了
images_test, labels_test = cifar10_input.inputs(eval_data=True,
                                                data_dir=data_dir,
                                                batch_size=batch_size)                                                  

### 2、模型构建

In [6]:
# 返回一个初始化w的同时，把正则项加入collection，wl是正则化系数
def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name='weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [7]:
# 计算总的损失函数
def loss(logits, labels):
#      """Add L2Loss to all the trainable variables.
#      Add summary for "Loss" and "Loss/avg".
#      Args:
#        logits: Logits from inference().
#        labels: Labels from distorted_inputs or inputs(). 1-D tensor
#                of shape [batch_size]
#      Returns:
#        Loss tensor of type float.
#      """
#      # Calculate the average cross entropy loss across the batch.
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

  # The total loss is defined as the cross entropy loss plus all of the weight
  # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')
 

In [8]:
# 模型建立，输入X，返回最后的输出
def inference(X):


    weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, wl=0.0)
    kernel1 = tf.nn.conv2d(X, weight1, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
    conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                       padding='SAME')
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)


    weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2, wl=0.0)
    kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                       padding='SAME')

    reshape = tf.reshape(pool2, [batch_size, -1])
    dim = reshape.get_shape()[1].value
    weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)
    bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
    local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

    weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)
    bias4 = tf.Variable(tf.constant(0.1, shape=[192]))                                      
    local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

    weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192.0, wl=0.0)
    bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
    logits = tf.add(tf.matmul(local4, weight5), bias5)
     
    return logits

### 3、模型训练

In [9]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

logits = inference(X= image_holder)
losses = loss(logits,label_holder)

train_op = tf.train.AdamOptimizer(1e-3).minimize(losses) #0.72

# 表示每个样本中，预测的前k大的概率中，是否有标签判断对了，有的话为True。这里k=1
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# 启动图片数据增强的线程队列，16线程加速，必须启动。
tf.train.start_queue_runners()
###
for step in range(max_steps):
    start_time = time.time()
    image_batch,label_batch = sess.run([images_train,labels_train])
    _, loss_value = sess.run([train_op, losses],feed_dict={image_holder: image_batch, 
                                                         label_holder:label_batch})
    duration = time.time() - start_time

    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)
    
        format_str = ('step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))
    
###

step 0, loss = 4.68 (35.4 examples/sec; 3.614 sec/batch)
step 10, loss = 3.72 (204.5 examples/sec; 0.626 sec/batch)
step 20, loss = 3.01 (203.3 examples/sec; 0.630 sec/batch)
step 30, loss = 2.69 (195.9 examples/sec; 0.654 sec/batch)
step 40, loss = 2.53 (207.1 examples/sec; 0.618 sec/batch)
step 50, loss = 2.28 (198.3 examples/sec; 0.645 sec/batch)
step 60, loss = 2.08 (200.8 examples/sec; 0.637 sec/batch)
step 70, loss = 2.07 (167.0 examples/sec; 0.766 sec/batch)
step 80, loss = 1.93 (196.2 examples/sec; 0.652 sec/batch)
step 90, loss = 2.09 (203.1 examples/sec; 0.630 sec/batch)
step 100, loss = 1.94 (198.0 examples/sec; 0.647 sec/batch)
step 110, loss = 1.98 (199.8 examples/sec; 0.641 sec/batch)
step 120, loss = 1.83 (199.3 examples/sec; 0.642 sec/batch)
step 130, loss = 1.75 (197.8 examples/sec; 0.647 sec/batch)
step 140, loss = 1.75 (208.8 examples/sec; 0.613 sec/batch)
step 150, loss = 1.85 (198.9 examples/sec; 0.644 sec/batch)
step 160, loss = 1.65 (208.3 examples/sec; 0.615 sec

### 4、模型测试

In [12]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0  
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch,label_batch = sess.run([images_test,labels_test])
    predictions = sess.run([top_k_op],feed_dict={image_holder: image_batch,
                                                 label_holder:label_batch})
    true_count += np.sum(predictions)
    step += 1

precision = true_count *1.0 / total_sample_count
print('precision @ 1 = %.3f' % precision)

precision @ 1 = 0.479
